# Model-fitting Cookbook

## Worked Example

### Fitting a line with maximum-likelihood methods

In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
plt.rcParams['xtick.labelsize'] = 'x-large'
plt.rcParams['ytick.labelsize'] = 'x-large'
%matplotlib inline

Here, we will walk through answering the basic model-fitting questions for a simple case:
1. What are the best-fitting parameter values?
2. What are the uncertainties on those values?
3. Is the model an acceptable fit to the data?

All of the code needed to run through is given below, but isn't explained in gruesome detail, so it's a good idea to check that you understand what's being done. I also make no claims about the code being in any sense optimal.

We'll use the classic example of fitting a line to data with Gaussian measurement uncertainties. To be precise,
* We're given a set of perfectly known $x_i$ values where data are measured.
* Our model says there is a single _true_ value of $y_i$ for each $x_i$, forming a line, $y_\mathrm{true}(x)=a+bx$.
* Our measured values, $y_i$, are assumed to be generated by independent Gaussian distributions with means $y_\mathrm{true}(x_i)$ and perfectly known standard deviations, $\sigma_i$.

To reiterate,
* $a$ and $b$ are our model parameters, which we want to fit
* $y_i$ are data, whose random generation is described by our model
* $x_i$ and $\sigma_i$ are also data in the sense of not being parameters we can vary, but we're assuming they're known perfectly.

That last point contains a pretty serious assertion, which isn't always warranted. But, at least for this example, let's roll with it.

This is a linear (in the parameters) model with a Gaussian likelihood, so we could use least squares methods to instantly get an exact solution.

(Aside: note that "linear in the parameters", not "linear in $x$", is what matters here. So, $y_\mathrm{true}(x)=a+bx+cx^2$ is also a linear least squares problem.)

Instead, let's use the more general maximum-likelihood approach, and check our results against least squares at the end.

This code simply generates (reproducibly) some fake data for us to fit. It defines $x$, $y$ and $\sigma$ values for us in the arrays `xs`, `ys` and `sigmas`. (It also defines an ellipse-plotting function we'll use to visualize the least-squares solution later.)

In [ ]:
from line_example import *

Let's take a look.

In [ ]:
plt.errorbar(xs, ys, yerr=sigmas, fmt='o');
plt.xlabel('x', fontsize=14);
plt.ylabel('y', fontsize=14);

According to the model assumptions defined in the bullet points above, our likelihood function should be

$\mathcal{L}\left(a,b;\{x_i,y_i,\sigma_i\}\right) = \prod_i \frac{1}{\sqrt{2\pi}\sigma_i} \exp \left[ -\frac{(a+bx_i - y_i)^2}{2\sigma_i^2} \right]$.

Conventionally, we work instead with the quantity $S = -2\ln(\mathcal{L})+\mathrm{const}$, where the "const" allows us to throw away terms that don't depend on the model parameters. We'll see later that only _differences_ in $S$ are meaningful.

In this case, because $\sigma_i$ are fixed, we're welcome to neglect the normalization factor in front of the exponential.

So, here's the log-likelihood function we'll use, which you may recognize as the $\chi^2$ function for this problem, $\chi^2 = \sum_i \frac{(a+bx_i-y_i)^2}{\sigma_i^2}$.

In [ ]:
def S(a, b):
    """
    -2*ln(likelihood) + arbitrary constant
    """
    y_mean = a + b*xs
    return np.sum( (ys - y_mean)**2 / sigmas**2 )

Now for some computational details. Our best-fitting parameters will be those that minimize $S(a,b)$. In general, it's not a bad idea to use one of the fancy multidimensional minimizers in `scipy` for this. But since we have such a simple function of only 2 parameters here, we can get away with a more brute-force evaluation on a grid.

This defines a course grid for our initial search, with extremely generous bounds based on inspection of the $x,y$ plot above.

In [ ]:
a_min = 0.0
a_max = 100.0
da = 10.0
b_min = 0.0
b_max = 2.0
db = 0.1
grid_b, grid_a = np.meshgrid(np.arange(b_min, b_max+db, db), np.arange(a_min, a_max+da, da))

It's never a bad idea to make sure we know which index corresponds to what parameter.

In [ ]:
plt.rcParams['figure.figsize'] = (14.0, 5.0)
fig, ax = plt.subplots(1,2);
ax[0].imshow(grid_a.T, cmap='gray', origin='lower', extent=[a_min, a_max, b_min, b_max], aspect='auto');
ax[0].set_xlabel('a', fontsize=14);
ax[0].set_ylabel('b', fontsize=14);
ax[1].imshow(grid_b.T, cmap='gray', origin='lower', extent=[a_min, a_max, b_min, b_max], aspect='auto');
ax[1].set_xlabel('a', fontsize=14);
ax[1].set_ylabel('b', fontsize=14);

A python detail: to evaluate $S$ over the whole grid in one line, we need to create a vectorized version of it. This is because `S` already includes vectorized evaluations involving $x$, $y$ and $\sigma$, and we don't want these conflated with vectorization over the grid of parameter values. (As written, `S` works as indended only if the arguments `a` and `b` are scalar.)

In [ ]:
# this is supposed to be no more efficient than a for loop,
# but could still be more convenient
vectorized_S = np.vectorize(S)

Let's now evaluate $S$ over the grid and see what that looks like:

In [ ]:
grid_S = vectorized_S(grid_a, grid_b)

plt.rcParams['figure.figsize'] = (6.0, 6.0)
plt.imshow(grid_S.T, origin='lower', aspect='auto', extent=[a_min, a_max, b_min, b_max]);
plt.xlabel('a', fontsize=14); plt.ylabel('b', fontsize=14);

Now we can find the values of $a$ and $b$ that minimize $S$ (at least among those on the grid).

In [ ]:
i = np.unravel_index(np.argmin(grid_S), grid_S.shape)
best_params = np.array((grid_a[i], grid_b[i]))
best_params

We'll do a quantitative goodness of fit test (far) below, but it's _always_ a good idea to do a quick, visual comparison of the fitted model with the data:

In [ ]:
plt.errorbar(xs, ys, yerr=sigmas, fmt='o', label='data');
plt.plot(np.sort(xs), best_params[0]+best_params[1]*np.sort(xs), '-', label='best fit');
plt.xlabel('x', fontsize=14); plt.ylabel('y', fontsize=14); plt.legend(loc='upper left');

Next, we may want to find marginalized confidence intervals for $a$ and $b$.

To do that we'll need _profile likelihoods_ for each parameter. Recall that these are univariate functions of each parameter:

$S_a(a) = \min_b S(a,b)$

$S_b(b) = \min_a S(a,b)$

In [ ]:
profS_a = np.min(grid_S, axis=1)
profS_b = np.min(grid_S, axis=0)

We also need to know what thresholds in $\Delta S = S - S_\mathrm{min}$ define the ends of a given confidence interval.

The most common confidence levels to report correspond to the area enclosed within $n\sigma$ of the mean of a Gaussian. There is no good reason for this, and non-physicists have mostly abandoned the convention. But you'll get used to it.

Here is how you can compute the confidence level corresponding to e.g. 1, 2 and 3 $\sigma$.

In [ ]:
st.chi2.cdf(np.arange(1,4)**2, 1)

To get the corresponding $\Delta S$ threshold for the corresponding confidence interval or region for $m$ parameters, we would evaluate the evaluate the _quantile function_ of the $\chi^2$ distribution with $m$ degrees of freedom at the desired confidence level. (Don't worry about the jargon.)

Here, we want $m=1$ (confidence intervals for just $a$ and just $b$, separately), so this gives us back the simple result that the $\Delta S$ corresponding to an $n\sigma$ confidence interval is $n^2$. _For joint confidence regions of multiple parameters, this will be different_.

In [ ]:
S_min = np.min(grid_S)
dS_levels1 = st.chi2.ppf(st.chi2.cdf(np.arange(1,3)**2, 1), 1)
dS_levels1

Let's plot the two profile log-likelihoods, with horizontal lines at the $1\sigma$ and $2\sigma$ $\Delta S$ thresholds:

In [ ]:
plt.rcParams['figure.figsize'] = (14.0, 5.0)
fig, ax = plt.subplots(1,2);
ax[0].plot(grid_a[:,0], profS_a, 'o-');
ax[0].plot(grid_a[:,0], grid_a[:,0]*0.0+S_min+dS_levels1[0], 'k--');
ax[0].plot(grid_a[:,0], grid_a[:,0]*0.0+S_min+dS_levels1[1], 'k--');
ax[0].set_xlabel('a', fontsize=14);
ax[0].set_ylabel('profile -2 log likelihood', fontsize=14);
ax[1].plot(grid_b[0,:], profS_b, 'o-');
ax[1].plot(grid_b[0,:], grid_b[0,:]*0.0+S_min+dS_levels1[0], 'k--');
ax[1].plot(grid_b[0,:], grid_b[0,:]*0.0+S_min+dS_levels1[1], 'k--');
ax[1].set_xlabel('b', fontsize=14);
ax[1].set_ylabel('profile -2 log likelihood', fontsize=14);

These plots make clear that my original choice of grid size and resolution was not good. The grid extends way beyond the range of $S$ values that we care about, and has hardly any points in the region we do care about. Not to mention, our best-fit parameter estimates can be no more accurate than the grid resolution.

We'll go back and re-run things with a better grid later, but for now let's press on and see how we would determine the _joint confidence region_ for $a$ and $b$.

As promised, we will need numerically different $\Delta S$ thresholds for these 2-dimensional confidence regions.

In [ ]:
dS_levels2 = st.chi2.ppf(st.chi2.cdf(np.arange(1,3)**2, 1), 2)
dS_levels2

We can visualize the confidence regions by putting a contour plot with the right levels over our grid.

In [ ]:
plt.rcParams['figure.figsize'] = (6.0, 6.0)
plt.imshow(grid_S.T, cmap='gray', origin='lower', aspect='auto', extent=[a_min, a_max, b_min, b_max]);
plt.contour(grid_a, grid_b, grid_S, colors='c', levels=S_min+dS_levels2);
plt.plot(best_params[0], best_params[1], 'or');
plt.xlabel('a', fontsize=14); plt.ylabel('b', fontsize=14);

Ok, it's hopefully clear once again that we need a finer grid for our results here to have any meaning, and that we can get away with making its extent much smaller.

Go ahead and use the cells below to repeat the calculations above with a more sensible grid.

In [ ]:
a_min = 15.0
a_max = 45.0
da = 0.1
b_min = 0.25
b_max = 0.65
db = 0.001
grid_b, grid_a = np.meshgrid(np.arange(b_min, b_max+db, db), np.arange(a_min, a_max+da, da))

grid_S = vectorized_S(grid_a, grid_b)
S_min = np.min(grid_S)

i = np.unravel_index(np.argmin(grid_S), grid_S.shape)
best_params = np.array((grid_a[i], grid_b[i]))
best_params

In [ ]:
plt.errorbar(xs, ys, yerr=sigmas, fmt='o', label='data');
plt.plot(np.sort(xs), best_params[0]+best_params[1]*np.sort(xs), '-', label='best fit');
plt.xlabel('x', fontsize=14); plt.ylabel('y', fontsize=14); plt.legend(loc='upper left');

In [ ]:
profS_a = np.min(grid_S, axis=1)
profS_b = np.min(grid_S, axis=0)

plt.rcParams['figure.figsize'] = (14.0, 5.0)
fig, ax = plt.subplots(1,2);
ax[0].plot(grid_a[:,0], profS_a, '-');
ax[0].plot(grid_a[:,0], grid_a[:,0]*0.0+S_min+dS_levels1[0], 'k--');
ax[0].plot(grid_a[:,0], grid_a[:,0]*0.0+S_min+dS_levels1[1], 'k--');
ax[0].set_xlabel('a', fontsize=14);
ax[0].set_ylabel('profile -2 log likelihood', fontsize=14);
ax[1].plot(grid_b[0,:], profS_b, '-');
ax[1].plot(grid_b[0,:], grid_b[0,:]*0.0+S_min+dS_levels1[0], 'k--');
ax[1].plot(grid_b[0,:], grid_b[0,:]*0.0+S_min+dS_levels1[1], 'k--');
ax[1].set_xlabel('b', fontsize=14);
ax[1].set_ylabel('profile -2 log likelihood', fontsize=14);

In [ ]:
plt.rcParams['figure.figsize'] = (6.0, 6.0)
plt.imshow(grid_S.T, cmap='gray', origin='lower', aspect='auto', extent=[a_min, a_max, b_min, b_max]);
plt.contour(grid_a, grid_b, grid_S, colors='c', levels=S_min+dS_levels2);
plt.plot(best_params[0], best_params[1], 'or');
plt.xlabel('a', fontsize=14);
plt.ylabel('b', fontsize=14);

Much nicer looking! We can now summarize our fit results in the conventional way, using the best-fitting values and "upper" and "lower" error bars giving the distance from the best fit to the edges of the marginalized $1\sigma$ confidence interval for each parameter:

In [ ]:
print('a =', best_params[0], grid_a[np.where(profS_a <= S_min+dS_levels1[0])[0][[0,-1]],0] - best_params[0])
print('b =', best_params[1], grid_b[0,np.where(profS_b <= S_min+dS_levels1[0])[0][[0,-1]]] - best_params[1])

Let's now check these constraints against what we would get from least squares. Again, for this (specific) problem, least squares is exact rather than approximate.

It turns out that `numpy` and `scipy` lack a simple least squares solver, which would do some linear algebraic calculations to leap straight to the correct answer.

So, instead, we'll use the approach more typical of nonlinear least squares, by iteratively searching for the minimum of $S$.

The `curve_fit` function used here is not fully general; it assumes the likelihood is a product of independent Gaussians with known width (as we have in this problem).

It returns the position of the minimum of $S$ (the best fit) and an estimate of the covariance of the parameters, based on the second derivatives of $S$ about the minimum. These would also pop out of a linear least squares routine.

In [ ]:
from scipy.optimize import curve_fit

def func(x, a, b):
    return a + b*x

popt, pcov = curve_fit(func, xs, ys, sigma=sigmas, absolute_sigma=True)

For a linear least squares problem (and only asymptotically for others), the marginalized confidence intervals
1. are symmetric about the best fit (cf our solution above)
2. are $\pm n$ times the square-root of the diagonal of the parameter covariance matrix (for $n\sigma$ confidence)

In [ ]:
print('a =', popt[0], '+-', np.sqrt(pcov[0,0]))
print('b =', popt[1], '+-', np.sqrt(pcov[1,1]))

Let's compare the joint confidence regions implied by this covariance matrix to those from the grid method:

In [ ]:
plt.rcParams['figure.figsize'] = (6.0, 6.0)
plt.imshow(grid_S.T, cmap='gray', origin='lower', aspect='auto', extent=[a_min, a_max, b_min, b_max]);
plt.contour(grid_a, grid_b, grid_S, colors='c', levels=S_min+dS_levels2);
ellipse(pcov, center=popt, level=st.chi2.cdf(1.0,1), ax=plt, fmt='ro', npts=25)
ellipse(pcov, center=popt, level=st.chi2.cdf(2.0**2,1), ax=plt, fmt='ro', npts=25)
plt.xlabel('a', fontsize=14);
plt.ylabel('b', fontsize=14);

Practical aside: if you have $>2$ parameters and/or your likelihood is slow to evaluate, it's probably best to use an iterative minimizer from the start to find the best fitting parameters, and estimate their covariance. You can then do a grid evaluation limited to a sensible region to find more exact confidence regions, instead of using a grid to narrow down on the best fit as we did here.

Keep in mind that you'd be doing a minimization over all but 2 parameters for every grid point in a plane that you want to find confidence regions in. This gets old quickly.

With enough free parameters (or an expensive enough likelihood), it can be more efficient to use Monte Carlo methods for the confidence region exploration. These are more closely associated with Bayesian analysis, so we won't go into them more here.

Finally, let's turn to the last of our original questions: we have the best fitting _line_ for our data, but is it actually a good fit?

Generally, we define a well fitting model as one that could plausibly produce the observed data.

One way to quantify this is to generate many mock data sets from the best-fitting model, and compute a descriptive statistic for each. If the same statistic computed on the real data is within this distribution, the mock data are "similar enough" to the real data that the best-fitting model can explain them.

A common choice of descriptive statistic is the $S_\mathrm{min}$ obtained by fitting each mock data set as if it were the real data.

Let's do this for our example:

In [ ]:
predicted_Smin = np.empty(10000)

for i in range(len(predicted_Smin)):
    ysim = np.random.normal(best_params[0]+best_params[1]*xs, sigmas)
    simopt, simcov = curve_fit(func, xs, ysim, sigma=sigmas, absolute_sigma=True)
    predicted_Smin[i] = np.sum( (simopt[0]+simopt[1]*xs - ysim)**2 / sigmas**2 )

Here's how the distribution of simulated $S_\mathrm{min}$ values compares with the real one.

In [ ]:
plt.rcParams['figure.figsize'] = (7.0, 5.0)
plt.hist(predicted_Smin, bins=50, histtype='step', label='simulations', color='b', linewidth=2);
plt.axvline(S_min, color='k', ls='--', label='real data');
plt.xlabel(r'$\chi^2_{min}$', fontsize=14);
plt.ylabel('Frequency', fontsize=14);
plt.legend();

We would make this quantitative by looking at the fraction of simulated values that are less than the $S_\mathrm{min}$ of the real data. Typically, if this fraction is $<0.01$ or $>0.99$ (i.e. the real value is far out in one of the tails of the distribution), we would say the fit is clearly bad, and re-evaluate our model assumptions.

For linear least squares, the distribution of $S_\mathrm{min}$ (which is $\chi^2_\mathrm{min}$ for that case) is known from first principles, so we don't actually have to carry out the simulation procedure. It's the $\chi^2_\nu$ distribution, with degrees of freedom ($\nu$) equal to the number of data points minus the number of free model parameters.

Observe:

In [ ]:
plt.rcParams['figure.figsize'] = (7.0, 5.0)
plt.hist(predicted_Smin, bins=50, histtype='step', label='simulations', density=True, color='b', linewidth=2);
xx = np.arange(0.0, 50.0, 1.0);
plt.plot(xx, st.chi2.pdf(xx, len(ys)-len(best_params)), 'r-', label='exact');
plt.axvline(S_min, color='k', ls='--', label='real data');
plt.xlabel(r'$\chi^2_{min}$', fontsize=14);
plt.ylabel('Density', fontsize=14);
plt.legend();

The mean of the $\chi^2_\nu$ distribution is $\nu$, hence the common wisdom that the reduced $\chi^2$, $\chi^2_\mathrm{min}/\nu$, should be "close to 1" for a good fit.

In our case the fit is fine (and we know for a fact that the data came from a linear model), even though the reduced $\chi^2$ isn't especially "close to 1". This just underscores that one should really compare $S_\mathrm{min}$ with the _distribution_ predicted by the model, instead of waving hands about closeness to 1.

In [ ]:
S_min/(len(ys)-len(best_params))

We could go on... but that seems like plenty already! Instead, why not put these maximum likelihood methods to the test? The next notebook outlines a different problem - one that doesn't conveniently map onto least squares.